In [4]:
import numpy as np
from dataset.mnist import load_mnist

# 交叉熵误差
$E=-\Sigma t_k log(y_K)$

In [3]:
def cross_entropy_error(y, t):
    delta = 1e-7 # 保护机制, 防止log0
    return -np.sum(t * np.log(y + delta))

# mini-batch

使用训练数据进行学习，严格来说， 就是针对训练数据计算损失函数的值，找出使该值尽可能小的参数

如果以全部数据为对象 求损失函数的和，则计算过程需要花费较长的时间

我们从全部数据中选出一部分，作为全部数据的近似。神经网络的学习也是从训练数据中选出一批数据（称为 mini-batch, 小 批量），然后对每个 mini-batch 进行学习

In [7]:
(x_train, t_train), (x_test, t_test) = \
load_mnist(normalize=True, one_hot_label=True)

train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]